## ORM

In [4]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

#### Many to One

In [16]:
engine = create_engine("sqlite:///many2one_test.db")
Base = declarative_base()

In [17]:
class Tevas(Base):
    __tablename__ = "tevas"
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    vaikas_id = Column(Integer, ForeignKey('vaikas.id'))
    vaikas = relationship("Vaikas")

class Vaikas(Base):
    __tablename__ = "vaikas"
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    mokymo_istaiga = Column("Mokymo įskaita", String)

Base.metadata.create_all(engine)

In [18]:
Session = sessionmaker(bind=engine)
session = Session()

`C`RUD

In [21]:
vaikas = Vaikas(vardas='Vaikas', pavarde="Tevaika", mokymo_istaiga="Ciurlionio Gimnazija")
tevas = Tevas(vardas="Tevas", pavarde="Tevaika", vaikas=vaikas)
session.add(tevas)
session.commit()

C`R`UD

In [27]:
vaikas = Vaikas(vardas="Naujas vaikas", pavarde="Tevaika")
tevas = session.query(Tevas).get(1)
tevas.vaikas = vaikas
session.commit()

In [42]:
tevas = session.query(Tevas).all()
tevas[0].vaikas

In [43]:
vaikas = session.query(Vaikas).all()

In [44]:
print([v.__dict__ for v in vaikas])

[{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x122c96ce0>, 'id': 1, 'pavarde': 'Tevaika', 'mokymo_istaiga': 'Ciurlionio Gimnazija', 'vardas': 'Vaikas'}, {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x122621ab0>, 'id': 2, 'pavarde': 'Tevaika', 'mokymo_istaiga': None, 'vardas': 'Naujas vaikas'}]


Pakeisti tevo vaiko duomenis

In [45]:
tevas = session.query(Tevas).get(1)
tevas.vaikas.pavarde = "NaujaPavarde"
session.commit()

In [46]:
tevas = session.query(Tevas).get(1)
tevas.vaikas.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x122621ab0>,
 'id': 2,
 'vardas': 'Naujas vaikas',
 'mokymo_istaiga': None,
 'pavarde': 'NaujaPavarde'}

In [48]:
vaikas = session.query(Vaikas).all()
print([v.pavarde for v in vaikas])


['Tevaika', 'NaujaPavarde']


CRU`D`

In [49]:
tevas = session.query(Tevas).get(1)
session.delete(tevas)
session.commit()

In [55]:
vaikas = session.query(Vaikas).all()


In [56]:
print([''])

[<__main__.Vaikas at 0x122622140>, <__main__.Vaikas at 0x1226843d0>]

In [57]:
tevas = session.query(Tevas).all()


In [59]:
vaikas = session.query(Vaikas).all()


In [60]:
vaikas

[<__main__.Vaikas at 0x122622140>, <__main__.Vaikas at 0x1226843d0>]

### One To Many

In [61]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

engine = create_engine('sqlite:///one2many_test.db')
Base = declarative_base()


class Tevas(Base):
    __tablename__ = "tevas"
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    vaikai = relationship("Vaikas", back_populates="tevas")


class Vaikas(Base):
    __tablename__ = "vaikas"
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    mokymo_istaiga = Column("Mokymo įskaita", String)
    tevas_id = Column(Integer, ForeignKey("tevas.id"))
    tevas = relationship("Tevas", back_populates="vaikai")

Base.metadata.create_all(engine)


In [62]:
Session = sessionmaker(bind=engine)
session = Session()

### Kaip įrašyti tėvą ir jo vaikus (Crud)

In [63]:
vaikas = Vaikas(vardas="Vaikas", pavarde="Vaikaitis")
vaikas2 = Vaikas(vardas="Vaikas 2", pavarde="Vaikaitis 2")
tevas = Tevas(vardas="Tevas", pavarde="Vaikaitis")
tevas.vaikai.append(vaikas)
tevas.vaikai.append(vaikas2)
session.add(tevas)
session.commit()

### Kaip nuskaityti tėvo vaikus (cRud)

In [64]:
tevas = session.query(Tevas).get(1)

In [67]:
for vaikas in tevas.vaikai:
    print(vaikas.vardas, vaikas.pavarde)

Vaikas Vaikaitis
Vaikas 2 Vaikaitis 2


### Kaip redaguoti tėvo vaikus (crUd)

In [68]:
tevas = session.query(Tevas).get(1)

In [70]:
tevas.vaikai[0].vardas = "Vaikas 1"
session.commit()

In [71]:
tevas = session.query(Tevas).get(1)
for vaikas in tevas.vaikai:
    print(vaikas.vardas)

Vaikas 1
Vaikas 2


In [72]:
vaikas = session.query(Vaikas).get(1)
print(vaikas.tevas.vardas)

Tevas


In [73]:
tevas = session.query(Tevas).get(1)
vaikas1 = tevas.vaikai[0]
tevas.vaikai.remove(vaikas1)
session.commit()

In [75]:
tevas = session.query(Tevas).get(1)
tevas.vaikai

### Many to Many

In [76]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey, create_engine, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [77]:
engine = create_engine('sqlite:///many2many_test.db')
Base = declarative_base()

In [78]:
association_table = Table('association', Base.metadata,
    Column('tevas_id', Integer, ForeignKey('tevas.id')),
    Column('vaikas_id', Integer, ForeignKey('vaikas.id'))
)

class Tevas(Base):
    __tablename__ = 'tevas'
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    vaikai = relationship("Vaikas", secondary=association_table, back_populates='tevai')

class Vaikas(Base):
    __tablename__ = 'vaikas'
    id = Column(Integer, primary_key=True)
    vardas = Column("Vardas", String)
    pavarde = Column("Pavardė", String)
    tevai = relationship("Tevas", secondary=association_table, back_populates='vaikai')

Base.metadata.create_all(engine)

In [79]:
Session = sessionmaker(bind=engine)
session = Session()

In [80]:
tevas1 = Tevas(vardas="Tėvas", pavarde="Tėvaika")
tevas2 = Tevas(vardas="Motina", pavarde="Tevienė")
vaikas1 = Vaikas(vardas="Vaikas", pavarde="Tėvaika")
vaikas2 = Vaikas(vardas="Vaikė", pavarde="Tėvaikytė")

tevas1.vaikai.append(vaikas1)
tevas2.vaikai.append(vaikas1)
tevas2.vaikai.append(vaikas2)

session.add(tevas1)
session.add(tevas2)
session.commit()

In [82]:
tevas = session.query(Tevas).get(2)
for vaikas in tevas.vaikai:
    print(vaikas.vardas, vaikas.pavarde)

Vaikas Tėvaika
Vaikė Tėvaikytė


In [83]:
vaikas = session.query(Vaikas).get(1)
for tevas in vaikas.tevai:
    print(tevas.vardas, tevas.pavarde)

Motina Tevienė
Tėvas Tėvaika


In [84]:
tevas = session.query(Tevas).get(2)
tevas.vaikai[0].vardas = "Vaikas 1"
session.commit()

In [86]:
tevas = session.query(Tevas).get(2)
vaikas1 = tevas.vaikai[0]
tevas.vaikai.remove(vaikas1)
session.commit()